In [63]:
#paquetes necesarios

import pandas as pd
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Importación de los datos

df = pd.read_excel('../data/raw/datos_crudos.xlsx', header = 1)

df = df[['Serial', 'Event', '&quot;Provincia&quot;', '&quot;CantÃ³n&quot;',
       '&quot;Distrito&quot;', '&quot;Location&quot;', 'Date (YMD)',
       'Comments', 'Cause', 'Description of Cause', 'Source', 'Magnitude', 'Duration (d)', 'Other sectors', 'Deaths', 'Injured', 'Missing', 'Houses Destroyed', 'Houses Damaged', 'Directly affected', 'Indirectly Affected', 'Relocated', 'Evacuated', 'Losses $Local', 'Education centers', 'Hospitals']]

df.columns = ['Serial', 'Event', 'Provincia', 'Canton',
       'Distrito', 'Ubicacion', 'Date (YMD)',
       'Comments', 'Cause', 'Description of Cause', 'Source', 'Magnitude', 'Duration (d)', 'Other sectors', 'Deaths', 'Injured', 'Missing', 'Houses Destroyed', 'Houses Damaged', 'Directly affected', 'Indirectly Affected', 'Relocated', 'Evacuated', 'Losses $Local', 'Education centers', 'Hospitals']

### Análisis Exploratorio de Datos

,Serial,Event,Provincia,Canton,Distrito,Ubicacion,Date (YMD),Comments,Cause,Description of Cause,...,Missing,Houses Destroyed,Houses Damaged,Directly affected,Indirectly Affected,Relocated,Evacuated,Losses $Local,Education centers,Hospitals
0,2019-00101,RAIN,GUANACASTE,SANTA CRUZ,TAMARINDO,Villarreal,2019-09-27 00:00:00,Viviendas: Hubo afectaciÃ³n por las fuertes ll...,ATMOSPHCONDITION,Sistema de baja presiÃ³n forma parte de zona d...,...,0,0,3,0,0,0,7,0,0,0.0
1,2019-00100,RAIN,CARTAGO,CARTAGO,OCCIDENTAL,Cartago Centro,2019-09-27 00:00:00,Cerca de 40 incedentes provocados por lluvias ...,ATMOSPHCONDITION,Sistema de baja presiÃ³n forma parte de zona d...,...,0,0,0,0,0,0,0,0,0,0.0
2,2019-00099,FLOOD,ALAJUELA,GUATUSO,SAN RAFAEL,San Francisco,2019-09-07 00:00:00,Viviendas: Fuertes lluvias y suelos saturados ...,ATMOSPHCONDITION,Zona de Convergencia e ingreso humedad del Pac...,...,0,0,0,0,0,0,15,0,0,0.0
3,2019-00098,STRONGWIND,ALAJUELA,ALAJUELA,TAMBOR,Colonia del RÃ­o,2019-08-30 00:00:00,CaÃ­da de Ã¡rbol sobre vivienda deja una famil...,ATMOSPHCONDITION,Sistema de baja presiÃ³n forma parte de zona d...,...,0,0,1,0,0,0,0,0,0,0.0
4,2019-00097,FLOOD,ALAJUELA,UPALA,UPALA,Upala Centro,2019-07-08 00:00:00,Evacuaciones: 66 personas albergadas en CTP y ...,ATMOSPHCONDITION,Onda Tropical #11,...,0,0,0,0,0,0,66,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18636,1970-1,LANDSLIDE,HEREDIA,SARAPIQUI,PUERTO VIEJO,NaN,1970-01-09 00:00:00,No hay paso entre RÃ­o Ãngel y el SarapiquÃ­.,RAIN,Temporal,...,0,0,0,0,0,0,0,0,0,0.0
18637,1968-4,ERUPTION,GUANACASTE,TILARAN,TRONADORA,VolcÃ¡n Arenal,1968-07-29 00:00:00,NaN,UNKNOWN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
18638,1968-3,ERUPTION,GUANACASTE,TILARAN,ARENAL,VolcÃ¡n Arenal,1968-07-29 00:00:00,NaN,UNKNOWN,NaN,...,0,0,0,0,0,0,0,0,0,0.0
18639,1968-2,ERUPTION,ALAJUELA,SAN RAMON,PEÃ‘AS BLANCAS,El Castillo,1968-07-29 00:00:00,NaN,ERUPTION,VolcÃ¡n Arenal,...,0,0,0,0,0,0,0,0,0,0.0
